In [1]:
import numpy as np

In [2]:
from bs4 import BeautifulSoup
import spacy
import re

nlp = spacy.load('en')

def clean(corpus):
    transformed = []
    for document in corpus:
        raw_text = BeautifulSoup(htm_text).get_text()
        analyzed = nlp(raw_text)
        
        tokens = [token for token in analyzed if not token.is_stop]
        tokens = [token for token in tokens if not token.is_punct]
        tokens = [token.lemma_ for token in tokens if token.lemma_ != '-PRON-']
        tokens = [token.lower() for token in tokens]
        
        document = ' '.join(tokens)
        document = document.replace('\n', '')
        document = document.replace('\\', '')
        document = document.replace('.', '')
        document = document.replace('"', '')
        document = ' '.join([x for x in document.split(' ') if len(x) > 2])
        
        transformed.append(document)
    return transformed

In [3]:
train = pd.read_csv('../data/labeledTrainData.tsv', sep='\t')

X_saved = clean(np.load('../data/cleanedTrainData.npy'))
len(X_saved)

25000

In [4]:
import pandas as pd

csv = pd.read_csv('../data/labeledTrainData.tsv', sep='\t')
y   = pd.get_dummies(csv['sentiment']).values

y.shape

(25000, 2)

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(X_saved)
X = tokenizer.texts_to_sequences(X_saved)
X = pad_sequences(X)

len(X)

25000

In [6]:
X.shape[1]

678

In [7]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

model = keras.Sequential()

model.add(Embedding(2000, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 678, 128)          256000    
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 678, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
from sklearn.model_selection import train_test_split

np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

print(X_train.shape, y_train.shape)
print(X_test.shape,  y_test.shape)

(15000, 678) (15000, 2)
(10000, 678) (10000, 2)


In [ ]:
batch_size = 32

# find workarounds to speed training
model.fit(X_train, y_train, batch_size=batch_size, epochs=5, verbose=2)